# 割当問題

* 0-1整数計画問題として解く(解ベクトルxの各要素を0または1のみに限定したもの)
* PythonのソルバーであるPuLPを利用する
* 割り当てた時の目的関数が最小になる組み合わせを探す

<a href="https://colab.research.google.com/github/ToumaTanaka/Data_Science/blob/main/Mathematical_Optimization/Allocation_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 23.2 MB/s 


In [24]:
import pulp
import pandas as pd
from google.colab import files
import time

In [31]:
#shabanを配送ルート、jomuin_codeをその日の出勤ドライバーとする
#scoreは最小化したい値

df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,shaban,jomuin_code,score
0,0,1,1,94
1,1,1,2,66
2,2,1,3,69
3,3,1,4,81
4,4,1,5,66


In [32]:
df['Var'] = [pulp.LpVariable(f'x{df.shaban[L]}_{df.jomuin_code[L]}',cat="Binary") for L in df.index]
df

,Unnamed: 0,shaban,jomuin_code,score,Var
0,0,1,1,94,x1_1
1,1,1,2,66,x1_2
2,2,1,3,69,x1_3
3,3,1,4,81,x1_4
4,4,1,5,66,x1_5
...,...,...,...,...,...
295,295,15,16,74,x15_16
296,296,15,17,98,x15_17
297,297,15,18,95,x15_18
298,298,15,19,76,x15_19


## 分散を最小化するためのモデル

In [27]:
#最小値探索のモデルを定義
problem = pulp.LpProblem('割り当て問題', sense=pulp.LpMinimize)

#制約条件

#同じ配送ルートごとにデータをまとめる
#iには配送ルートの番号が格納、vには同じ配送ルートの配列が格納されている
#一つの配送ルートに必ず一人を割り当てる制約
for i, v in df.groupby('shaban'):
  problem += pulp.lpSum(v.Var) == 1
  


#同じドライバーごとにデータをまとめる
#jにはドライバーの番号が格納、vには同じドライバーの配列が格納されている
#一人のドライバーに一つ割り当てるか、または一つも割り当てない制約
for j, v in df.groupby('jomuin_code'):
  problem += pulp.lpSum(v.Var) <= 1


#定義したモデルに目的関数を追加
#df.scoreとdf.Varの内積
problem += pulp.lpDot(df.score,df.Var)

## 最大値を最小化するためのモデル


In [33]:
#変数を定義
z = pulp.LpVariable('z', lowBound=0) 

#最小値探索のモデルを定義
problem = pulp.LpProblem('割り当て問題', sense=pulp.LpMinimize)

#制約条件

#同じ配送ルートごとにデータをまとめる
#iには配送ルートの番号が格納、vには同じ配送ルートの配列が格納されている

#一つの配送ルートに必ず一人を割り当てる制約
for i, v in df.groupby('shaban'):
  problem += pulp.lpSum(v.Var) == 1
  


#同じドライバーごとにデータをまとめる
#jにはドライバーの番号が格納、vには同じドライバーの配列が格納されている
#一人のドライバーに一つ割り当てるか、または一つも割り当てない制約
for j, v in df.groupby('jomuin_code'):
  problem += pulp.lpSum(v.Var) <= 1


#合計がz以下になるような定式化
for j, v in df.groupby('jomuin_code'):
  problem += pulp.lpDot(df.score,df.Var) <= z


#定義したモデルに目的関数を追加
#zの最小化
problem += z

In [34]:
# 処理前の時刻
t1 = time.time() 

#解を求める
result = problem.solve()

# 処理後の時刻
t2 = time.time()

elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

#現在のstatusを表示
#-3: 'Undefined',    未定義
#-2: 'Unbounded',   非有界
#-1: 'Infeasible',     実行不可能
#0: 'Not Solved'    解けなかった
#1: 'Optimal'       最適解を発見
print(pulp.LpStatus[result])

#目的関数の値を表示
print(pulp.value(problem.objective))

#最適化結果をデータフレームに追加
df['Val'] = df.Var.apply(pulp.value)

経過時間：0.05791735649108887
Optimal
808.0


In [35]:
df.head()

,Unnamed: 0,shaban,jomuin_code,score,Var,Val
0,0,1,1,94,x1_1,0.0
1,1,1,2,66,x1_2,0.0
2,2,1,3,69,x1_3,0.0
3,3,1,4,81,x1_4,0.0
4,4,1,5,66,x1_5,0.0


In [36]:
df_s = df[df.Val > 0][['shaban', 'jomuin_code', 'score']]
df_s

,shaban,jomuin_code,score
18,1,19,63
34,2,15,57
56,3,17,50
64,4,5,50
90,5,11,54
102,6,3,56
127,7,8,54
152,8,13,51
168,9,9,50
191,10,12,60


In [ ]:
#結果の出力
df_s.to_csv("optimal.csv")
files.download("optimal.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>